In [1]:
using Rocket
using Distributions
using ReactiveMP
using BenchmarkTools

using Profile
using PProf
using ProfileSVG

import Base: show

┌ Info: Precompiling ReactiveMP [a194aa59-28ba-4574-a09c-4a745416d6e3]
└ @ Base loading.jl:1260
┌ Warning: Package ReactiveMP does not have Rocket in its dependencies:
│ - If you have ReactiveMP checked out for development and have
│   added Rocket as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with ReactiveMP
└ Loading Rocket into ReactiveMP from project dependency, future warnings for ReactiveMP are suppressed.


In [2]:
function createSubgraph(index::Int)
    noise_add = AdditionNode()
    
    noise = constvar(:noise, Normal(0.0, sqrt(200.0)))
    yn    = datavar(:yn, Float64)
    
    connect!(noise_add, :in2, noise, 1)
    connect!(noise_add, :out, yn, 1)
    
    return (yn, noise_add)
end

function createGraph(size::Int)    
    c0_add = AdditionNode()
    
    c0       = constvar(:c0, 1.0)
    x0_prior = datavar(:x0_prior, Normal{Float64})
    
    connect!(c0_add, :in1, x0_prior, 1)
    connect!(c0_add, :in2, c0, 1)
    
    index = 1
    
    prev_c_add = c0_add
    
    xs = Vector{RandomVariable}(undef, size)
    ys = Vector{DataVariable}(undef, size)
    
    while index < size
        yn, noise_add = createSubgraph(index)
        xn            = randomvar(:xn, 3)
        
        c_add = AdditionNode()
        cn    = constvar(:cn, 1.0)
        
        connect!(prev_c_add, :out, xn, 1)
        connect!(noise_add, :in1, xn, 2)
        connect!(c_add, :in1, xn, 3)
        connect!(c_add, :in2, cn, 1)
        
        activate!(prev_c_add)
        activate!(noise_add)
        
        xs[index] = xn
        ys[index] = yn
        
        prev_c_add = c_add
        
        index += 1
    end
    
    last_noise_add = AdditionNode();
    
    x_last     = randomvar(:x_last, 2)
    y_last     = datavar(:ylast, Float64)
    last_noise = constvar(:last_noise, Normal(0.0, sqrt(200.0)))
    
    connect!(prev_c_add, :out, x_last, 1)
    connect!(last_noise_add, :in1, x_last, 2)
    connect!(last_noise_add, :in2, last_noise, 1)
    connect!(last_noise_add, :out, y_last, 1)
    
    activate!(prev_c_add)
    activate!(last_noise_add)
    
    xs[size] = x_last
    ys[size] = y_last
    
    return (xs, ys, x0_prior)
end

createGraph (generic function with 1 method)

In [3]:
function smoothing(data)
    N = length(data)
    
    xs, ys, x_prior = createGraph(N);
    
    messages      = Vector{Normal{Float64}}(undef, N)
    subscriptions = Vector{Teardown}(undef, N)
    
    @inbounds for (index, x) in enumerate(xs)
        subscriptions[index] = subscribe!(getbelief(x), (d) -> messages[index] = getdata(d))
    end
    
    update!(x_prior, Normal(0.0, sqrt(10000.0)))
    for i in 1:N
       update!(ys[i], data[i])
    end
    
    foreach(unsubscribe!, subscriptions)
    
    return messages
end

smoothing (generic function with 1 method)

In [4]:
N = 600
data = collect(1:N) + sqrt(200.0) * randn(N);

In [7]:
@btime smoothing($data)

  56.008 ms (386360 allocations: 14.28 MiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=1.6509419996225692, σ=0.5773406469256954)
 Normal{Float64}(μ=2.6509419996225687, σ=0.5773406469256954)
 Normal{Float64}(μ=3.6509419996225687, σ=0.5773406469256954)
 Normal{Float64}(μ=4.650941999622568, σ=0.5773406469256953)
 Normal{Float64}(μ=5.650941999622568, σ=0.5773406469256954)
 Normal{Float64}(μ=6.650941999622568, σ=0.5773406469256954)
 Normal{Float64}(μ=7.650941999622568, σ=0.5773406469256954)
 Normal{Float64}(μ=8.65094199962257, σ=0.5773406469256954)
 Normal{Float64}(μ=9.650941999622567, σ=0.5773406469256954)
 Normal{Float64}(μ=10.650941999622566, σ=0.5773406469256953)
 Normal{Float64}(μ=11.650941999622566, σ=0.5773406469256954)
 Normal{Float64}(μ=12.650941999622566, σ=0.5773406469256953)
 Normal{Float64}(μ=13.650941999622567, σ=0.5773406469256954)
 ⋮
 Normal{Float64}(μ=589.6509419996225, σ=0.577340646925696)
 Normal{Float64}(μ=590.6509419996224, σ=0.577340646925696)
 Normal{Float64}(μ=591.6509419996225, σ=0.5773406469256

In [6]:
@time smoothing(data)

  0.069063 seconds (386.36 k allocations: 14.285 MiB)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=1.6509419996225692, σ=0.5773406469256954)
 Normal{Float64}(μ=2.6509419996225687, σ=0.5773406469256954)
 Normal{Float64}(μ=3.6509419996225687, σ=0.5773406469256954)
 Normal{Float64}(μ=4.650941999622568, σ=0.5773406469256953)
 Normal{Float64}(μ=5.650941999622568, σ=0.5773406469256954)
 Normal{Float64}(μ=6.650941999622568, σ=0.5773406469256954)
 Normal{Float64}(μ=7.650941999622568, σ=0.5773406469256954)
 Normal{Float64}(μ=8.65094199962257, σ=0.5773406469256954)
 Normal{Float64}(μ=9.650941999622567, σ=0.5773406469256954)
 Normal{Float64}(μ=10.650941999622566, σ=0.5773406469256953)
 Normal{Float64}(μ=11.650941999622566, σ=0.5773406469256954)
 Normal{Float64}(μ=12.650941999622566, σ=0.5773406469256953)
 Normal{Float64}(μ=13.650941999622567, σ=0.5773406469256954)
 ⋮
 Normal{Float64}(μ=589.6509419996225, σ=0.577340646925696)
 Normal{Float64}(μ=590.6509419996224, σ=0.577340646925696)
 Normal{Float64}(μ=591.6509419996225, σ=0.5773406469256

In [5]:
@time smoothing(data)

  6.218727 seconds (16.98 M allocations: 908.541 MiB, 4.04% gc time)


600-element Array{Normal{Float64},1}:
 Normal{Float64}(μ=1.6509419996225692, σ=0.5773406469256954)
 Normal{Float64}(μ=2.6509419996225687, σ=0.5773406469256954)
 Normal{Float64}(μ=3.6509419996225687, σ=0.5773406469256954)
 Normal{Float64}(μ=4.650941999622568, σ=0.5773406469256953)
 Normal{Float64}(μ=5.650941999622568, σ=0.5773406469256954)
 Normal{Float64}(μ=6.650941999622568, σ=0.5773406469256954)
 Normal{Float64}(μ=7.650941999622568, σ=0.5773406469256954)
 Normal{Float64}(μ=8.65094199962257, σ=0.5773406469256954)
 Normal{Float64}(μ=9.650941999622567, σ=0.5773406469256954)
 Normal{Float64}(μ=10.650941999622566, σ=0.5773406469256953)
 Normal{Float64}(μ=11.650941999622566, σ=0.5773406469256954)
 Normal{Float64}(μ=12.650941999622566, σ=0.5773406469256953)
 Normal{Float64}(μ=13.650941999622567, σ=0.5773406469256954)
 ⋮
 Normal{Float64}(μ=589.6509419996225, σ=0.577340646925696)
 Normal{Float64}(μ=590.6509419996224, σ=0.577340646925696)
 Normal{Float64}(μ=591.6509419996225, σ=0.5773406469256